# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `generative metadata format` (`gmf`) metadata file from a dataset using MetaSynth. We are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

You can run this notebook by checking out the MetaSynth repo and installing metasynth with `pip install .`

In [ ]:
%pip install ..

In [1]:
# import required packages
import datetime as dt
import pandas as pd
from metasynth import MetaDataset

## Step 1: Transforming your data into a pandas DataFrame

The first step in creating the metadata is reading and converting your dataset to a pandas DataFrame. 

In [2]:
df = pd.read_csv("demonstration.csv")
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"Braund, Mr. Owen Harris",male,22.0,0,7.2500,NaN,S,1940-04-11,14:45:01.074798,2022-08-09 12:28:29.982572
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,0,71.2833,C85,C,1908-11-02,18:29:47.187362,2022-07-31 23:19:10.604536
2,3,"Heikkinen, Miss. Laina",female,26.0,0,7.9250,NaN,S,1906-08-21,12:08:02.689844,2022-07-28 19:13:45.452067
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,0,53.1000,C123,S,1913-11-19,14:51:08.508986,2022-08-01 18:22:06.233514
4,5,"Allen, Mr. William Henry",male,35.0,0,8.0500,NaN,S,1906-02-02,12:12:28.321094,2022-07-16 02:50:44.237489


MetaSynth will automatically generate the metadata from this DataFrame object so it is important to __ensure the data types for all the variables are correct__. For example, in the dataset above we see that Age is a floating point number whereas it should be an integer (22 instead of 22.0). In addition, there are a few categorical variables (Sex, Parch, Embarked) which are loaded in as string data types.

In general, we support [pandas dtypes](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes). For our example dataset we can specify the `dtypes` and load the dataset as follows:

In [3]:
dtypes = {
    "Survived": "category",  # Categories should be assigned this type.
    "Name": "string",  # Strings should be assigned like this
    "Age": "Int64",  # Integer columns that have NA's in them should be explicitly nullable integers.
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category",
}
df = pd.read_csv("demonstration.csv", dtype=dtypes)
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"Braund, Mr. Owen Harris",male,22,0,7.2500,<NA>,S,1940-04-11,14:45:01.074798,2022-08-09 12:28:29.982572
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,0,71.2833,C85,C,1908-11-02,18:29:47.187362,2022-07-31 23:19:10.604536
2,3,"Heikkinen, Miss. Laina",female,26,0,7.9250,<NA>,S,1906-08-21,12:08:02.689844,2022-07-28 19:13:45.452067
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,0,53.1000,C123,S,1913-11-19,14:51:08.508986,2022-08-01 18:22:06.233514
4,5,"Allen, Mr. William Henry",male,35,0,8.0500,<NA>,S,1906-02-02,12:12:28.321094,2022-07-16 02:50:44.237489


### Dates, times, and datetimes

One exception to the above is dates, times, and datetimes. Here, we use the types from the built-in `datetime` package. Thus we have to manually transform the strings in the columns with date, time, and datetimes to their proper objects. Since the columns in our example dataset follow the standard ISO-format, we can convert them with the `fromisoformat` method. If they are written in a different format, check out the [datetime library documentation](https://docs.python.org/3/library/datetime.html) on how to convert the strings to datetime/time/date objects.

In [4]:
df["Birthday"] = [dt.date.fromisoformat(x) for x in df["Birthday"]]
df["Board time"] = [dt.time.fromisoformat(x) for x in df["Board time"]]
df["Married since"] = [dt.datetime.fromisoformat(x) for x in df["Married since"]]

Now, let's check the data types of our DataFrame:

In [5]:
df.dtypes

PassengerId               int64
Name                     string
Sex                    category
Age                       Int64
Parch                  category
Fare                    float64
Cabin                    string
Embarked               category
Birthday                 object
Board time               object
Married since    datetime64[ns]
dtype: object

We see that most variables are now nicely specified as strings, categories and ints where necessary. For the dates and times we just created, we see the dtype `object`. This is the "catch-all" dtype for pandas. But don't worry, these columns have the correct type and MetaSynth will deal with it correctly:

In [6]:
df["Birthday"][0]

datetime.date(1940, 4, 11)

## Step 2: Creating a MetaDataset object from a DataFrame

Now a lot of work has already gone into creating a properly formatted dataframe. This work pays off at this stage: let's convert the DataFrame to a meta_dataset structure with the default options. Note: this takes a little bit of time!

In [7]:
meta_dataset = MetaDataset.from_dataframe(df)

c:\Users\erikj\surfdrive\SoDa\projects\synthetic_data\meta-synth\venv\lib\site-packages\metasynth\disttree.py:119: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"


Then, we can show the metadata as a dictionary:

In [8]:
print(meta_dataset)

# Rows: 891
# Columns: 11

{'name': 'PassengerId', 'description': None, 'type': 'MetaVar', 'dtype': 'int64', 'prop_missing': 0.0, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Name', 'description': None, 'type': 'MetaVar', 'dtype': 'string', 'prop_missing': 0.0, 'distribution': '.[]{12,82}'}

{'name': 'Sex', 'description': None, 'type': 'MetaVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameters': {'labels': array(['female', 'male'], dtype='<U6'), 'probs': array([0.35241302, 0.64758698])}}"}

{'name': 'Age', 'description': None, 'type': 'MetaVar', 'dtype': 'Int64', 'prop_missing': 0.19865319865319866, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 0, 'high': 81}}"}

{'name': 'Parch', 'description': None, 'type': 'MetaVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameters': {'lab

## Step 3: Saving the metadata in a file

After creating the metadata, we can save it to a file. The default format is `json`, meaning the file is quite legible by humans and computers alike. Therefore, it can be checked by the data controller and, when the disclosure risk is deemed to be low, this file can be shared with others.

In [9]:
file_path = "demonstration_metadata.json"
meta_dataset.to_json(file_path)

## Step 4: Generating synthetic data from the metadata

Upon receiving this file, you can use the MetaSynth package to generate a synthetic version of the dataset:

In [12]:
new_meta_dataset = MetaDataset.from_json(file_path)
new_meta_dataset.synthesize(5)

ValidationError: 'labels' is a required property

Failed validating 'required' in schema[0]['properties']['parameters']:
    {'properties': {'labels': {'items': {'type': 'string'},
                               'type': 'array',
                               'uniqueItems': True},
                    'probs': {'items': {'type': 'number'},
                              'type': 'array'}},
     'required': ['labels', 'probs'],
     'type': 'object'}

On instance['parameters']:
    {'end': '2022-08-15T10:24:40.438330',
     'start': '2022-07-15T12:10:47.959210'}

As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

### Set unique columns

One column (PassengerId) has been detected as possibly unique by MetaSynth, as indicated by the following warning:

> "Variable PassengerId seems unique, but not set to be unique."

This column holds a variable with unique passenger identifiers, so in fact we do want synthetic data generated for this column to be unique as well. We can add this to the metadata by creating a list of options which we call a `specification`, or `spec`:

In [13]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}
}

# then, we add that dictionary as the `spec` argument
meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)

# then, let's check what the metadata about PassengerId contains!
meta_dataset["PassengerId"].to_dict()

{'name': 'PassengerId',
 'type': 'discrete',
 'dtype': 'int64',
 'prop_missing': 0.0,
 'distribution': {'name': 'UniqueKeyDistribution',
  'parameters': {'low': 1, 'consecutive': 1}}}

So let's check what is generated from this new metadata:

In [14]:
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,";Ss>HB ,F dRD8R_:zwFI_|aNz2yAMFy;^O)<gN",male,44,0,6.440097,<NA>,C,1913-01-17,13:02:36.102309,2022-07-16 23:23:19.527328439
1,2,"I2QdMe+z,VSnh""x9>VidM?hjb?< Yp<<Ywmb Yy:g)D%e ...",male,60,0,81.673826,<NA>,S,1912-06-02,12:17:25.317707,2022-07-30 16:56:36.526808807
2,3,"$,C&b+WK2[2#h`:'G{?D p:Un p|b1",male,72,0,19.791133,<NA>,S,1920-10-05,12:45:05.513515,2022-08-10 04:03:58.434168666
3,4,">EF:o4vi}4;""jk_SY28z'_6We3p}#*Ob&",female,33,2,23.008364,<NA>,S,1933-05-17,15:05:00.508080,2022-08-11 08:43:10.133107913
4,5,"x'+@T_ Aj&u\WSio!q}w,+QbdgmVeT <K`AqF5 zbX,5;d...",male,36,2,71.328421,<NA>,S,1920-08-16,10:57:33.348754,2022-07-25 07:47:03.996000852


### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in MetaSynth is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, MetaSynth supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We fake names as follows:

In [15]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Ashley Spencer,male,10,2,6.441751,F|b5H!,S,1927-12-29,12:03:05.203712,2022-08-14 03:42:30.692552587
1,2,Brandon Guerrero,male,60,0,35.803859,Cx4l^[Um#,S,1928-06-01,12:15:05.227432,2022-07-19 11:38:39.328851342
2,3,Joshua Brown,female,<NA>,2,159.817131,<NA>,S,1905-01-06,16:55:27.613031,2022-07-16 11:44:21.454787938
3,4,Jennifer Guzman,male,44,0,4.206496,Tc,C,1910-09-06,11:23:47.793923,2022-08-01 19:47:23.730898398
4,5,James Cabrera,male,17,0,3.176234,<NA>,S,1934-07-15,15:57:59.842304,2022-08-11 15:36:58.390180609


### Set distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the best fitting from available distributions for the variable type. However, we can also manually specify which distribution to fit, or we can even just fully specify how the variable should be generated.

In [16]:
from metasynth.distribution import DiscreteUniformDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)} # fully specify a distribution for age (uniform between 20 and 40)
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,James Sherman,female,<NA>,0,2.722461,DB/0%W,S,1919-05-20,14:02:04.409878,2022-07-27 23:06:50.575963779
1,2,Pamela Black,male,<NA>,1,0.246326,<NA>,Q,1916-07-30,12:52:20.649790,2022-08-06 05:11:18.924598945
2,3,Stephanie Hobbs,female,23,0,5.153517,<NA>,C,1914-05-02,12:57:13.290085,2022-07-25 22:39:15.605316780
3,4,Crystal Rhodes,male,39,0,1.672135,TO.Hm;1,S,1910-12-18,13:10:07.851304,2022-08-09 00:10:53.921616059
4,5,Karen Johnson,male,30,0,2.971407,<NA>,S,1920-12-30,12:16:03.199466,2022-07-31 01:42:01.482494066


### Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [18]:
from metasynth.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution([(r"[ABCDEF]", 1), (r"\d{2,3}", 0.9)])  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [ABCDEF]?(\d{2,3})?

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)}, # fully specify a distribution for age (uniform between 20 and 40)
    "Cabin": {"distribution": cabin_distribution}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Robert Williams PhD,male,<NA>,0,7.211377,<NA>,S,1921-10-09,14:21:39.327001,2022-08-02 09:40:07.111689271
1,2,Charles Bailey,male,30,0,0.245903,<NA>,S,1929-07-31,17:23:22.286751,2022-07-20 12:55:21.699786025
2,3,David Crawford,male,34,0,0.666758,F711,S,1926-05-10,13:52:59.084411,2022-07-21 08:22:12.922703969
3,4,Michelle Smith,female,36,0,3.317543,<NA>,C,1915-02-09,17:14:51.560241,2022-07-28 05:28:51.619423380
4,5,Andrew Perez,male,26,2,0.241524,<NA>,S,1924-06-06,12:51:00.517828,2022-08-06 11:37:36.365830545
